In [426]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [451]:
train_gen = ImageDataGenerator(
    rescale=1./255,               # rescale pixel values to [0, 1]
    rotation_range=10,            # rotates the image by 0 - 10 degrees (not much tilt in birds in the images)
    zoom_range=[0.3,0.7],         # zooms the image between 30% to 70% because the birds in the most of the images are small
    width_shift_range=0.5,        # randomly shift images horizontally by 50% of the width because the birds are not always in the centre of the image
    height_shift_range=0.2,       # randomly shift images vertically by 20% of the length (most of the birds of the images are close to the middle horizontally)
    horizontal_flip=True,         # randomly flip images horizontally
    brightness_range=[0.8, 1.2],  # randomly adjust brightness between 0.8 and 1.2
    fill_mode='nearest')          # Fill any newly created pixels with the nearest value)

test_gen = ImageDataGenerator(rescale=(1./255))  

train = train_gen.flow_from_directory('bird/train_data/train_data',
                                      target_size=(224,224), #since the birds are small, have to have a relatively big target size
                                      class_mode='categorical', 
                                      batch_size=10)
test = test_gen.flow_from_directory('bird/test_data/test_data',target_size=(224,224),
                                      class_mode='categorical', batch_size=10)


Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [428]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [429]:
# initialising cnn

from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Convolution2D(48,(3,3), activation='relu', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(Convolution2D(12,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(24,activation='relu'))
model.add(Dense(28,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='softmax'))

model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_103 (Conv2D)         (None, 222, 222, 48)      1344      
                                                                 
 batch_normalization_10 (Bat  (None, 222, 222, 48)     192       
 chNormalization)                                                
                                                                 
 max_pooling2d_70 (MaxPoolin  (None, 111, 111, 48)     0         
 g2D)                                                            
                                                                 
 conv2d_104 (Conv2D)         (None, 109, 109, 36)      15588     
                                                                 
 max_pooling2d_71 (MaxPoolin  (None, 54, 54, 36)       0         
 g2D)                                                            
                                                     

In [430]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [433]:
model.fit(train,batch_size=10,validation_data=test,epochs=50)

Epoch 1/50
15/15 [==============================] - 33s 2s/step - loss: 2.0931 - accuracy: 0.3467 - val_loss: 3.2699 - val_accuracy: 0.1146
Epoch 2/50
15/15 [==============================] - 33s 2s/step - loss: 2.1553 - accuracy: 0.2733 - val_loss: 2.9478 - val_accuracy: 0.1146
Epoch 3/50
15/15 [==============================] - 33s 2s/step - loss: 2.1079 - accuracy: 0.3200 - val_loss: 2.9699 - val_accuracy: 0.1274
Epoch 4/50
15/15 [==============================] - 33s 2s/step - loss: 2.0186 - accuracy: 0.2933 - val_loss: 2.6577 - val_accuracy: 0.1783
Epoch 5/50
15/15 [==============================] - 33s 2s/step - loss: 2.0207 - accuracy: 0.3333 - val_loss: 2.7998 - val_accuracy: 0.1592
Epoch 6/50
15/15 [==============================] - 33s 2s/step - loss: 1.9127 - accuracy: 0.3400 - val_loss: 2.5255 - val_accuracy: 0.2293
Epoch 7/50
15/15 [==============================] - 33s 2s/step - loss: 2.0351 - accuracy: 0.3467 - val_loss: 2.6463 - val_accuracy: 0.1529
Epoch 8/50
15/15 [==

In [434]:
#using transfer learning

In [452]:
# using VGG16

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))


In [453]:
#train model with existing weights

for layer in vgg.layers:
    print(layer)

In [454]:
#train model with existing weights

for layer in vgg.layers:
    layer.trainable=False      #it won't load weights at the start

In [455]:
x = Flatten()(vgg.output)      #output matrix to vector form

In [456]:
#output layer

prediction = Dense(16,activation='softmax')(x)

In [457]:
#create Vgg16 model
from tensorflow.keras.models import Model

model = Model(inputs=vgg.input,outputs=prediction)

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [458]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [460]:
model.fit_generator(train,validation_data=test,epochs=20,steps_per_epoch=len(train),validation_steps=len(test))

C:\Users\rahul\AppData\Local\Temp/ipykernel_14736/985259952.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test,epochs=20,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/20
15/15 [==============================] - 45s 3s/step - loss: 1.4231 - accuracy: 0.5933 - val_loss: 4.2022 - val_accuracy: 0.2293
Epoch 2/20
15/15 [==============================] - 46s 3s/step - loss: 1.4939 - accuracy: 0.5467 - val_loss: 4.0411 - val_accuracy: 0.2548
Epoch 3/20
15/15 [==============================] - 45s 3s/step - loss: 1.3697 - accuracy: 0.6267 - val_loss: 4.1106 - val_accuracy: 0.3439
Epoch 4/20
15/15 [==============================] - 45s 3s/step - loss: 1.3091 - accuracy: 0.6267 - val_loss: 4.1944 - val_accuracy: 0.2166
Epoch 5/20
15/15 [==============================] - 45s 3s/step - loss: 1.1728 - accuracy: 0.6200 - val_loss: 3.8864 - val_accuracy: 0.3248
Epoch 6/20
15/15 [==============================] - 46s 3s/step - loss: 1.1036 - accuracy: 0.6267 - val_loss: 4.2018 - val_accuracy: 0.3248
Epoch 7/20
15/15 [==============================] - 46s 3s/step - loss: 1.2049 - accuracy: 0.6533 - val_loss: 4.1221 - val_accuracy: 0.2866
Epoch 8/20
15/15 [==

In [443]:
# using ResNet50

from tensorflow.keras.applications.resnet50 import ResNet50

resnet = ResNet50(include_top=False,input_shape=(224,224,3))    #weights ='imagenet' is selected by default

In [444]:
for layer in resnet.layers:
    print(layer)

In [445]:
for layer in resnet.layers:
    layer.trainable=False

In [446]:
x = Flatten()(resnet.output)

In [447]:
out = Dense(16,activation='softmax')(x)

In [448]:
# create resnet50 model

res_model = Model(inputs=resnet.input,outputs=out)

res_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_8[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [449]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [450]:
res_model.fit(train,epochs=20,validation_data=test,steps_per_epoch=len(train),validation_steps=len(test))

Epoch 1/10
15/15 [==============================] - 42s 3s/step - loss: 19.9700 - accuracy: 0.1000 - val_loss: 20.6404 - val_accuracy: 0.0701
Epoch 2/10
15/15 [==============================] - 38s 3s/step - loss: 15.3338 - accuracy: 0.1200 - val_loss: 11.6528 - val_accuracy: 0.0446
Epoch 3/10
15/15 [==============================] - 38s 3s/step - loss: 7.2807 - accuracy: 0.1533 - val_loss: 7.2795 - val_accuracy: 0.1083
Epoch 4/10
15/15 [==============================] - 38s 3s/step - loss: 5.5708 - accuracy: 0.1467 - val_loss: 5.4430 - val_accuracy: 0.1019
Epoch 5/10
15/15 [==============================] - 38s 3s/step - loss: 5.3355 - accuracy: 0.1400 - val_loss: 6.7776 - val_accuracy: 0.1083
Epoch 6/10
15/15 [==============================] - 38s 3s/step - loss: 6.1832 - accuracy: 0.1133 - val_loss: 5.4172 - val_accuracy: 0.1720
Epoch 7/10
15/15 [==============================] - 38s 3s/step - loss: 4.4473 - accuracy: 0.1400 - val_loss: 5.0288 - val_accuracy: 0.1975
Epoch 8/10
15/15